In [ ]:
import simpy
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import genfromtxt
import pandas as pd
import scipy.stats as st
from tqdm import tqdm
plt.style.use('seaborn')

In [ ]:
def calculate_confidence_interval(matrix):
    """This function generates a 95% confidence interval for a matrix of areas calculated using MC simulations
    Args:
        matrix (numpy array 2D): matrix containing all area computations
    Returns:
        numpy array: array of confidence intervals for the average of each simulation
    """

    cis = np.ones(shape = (1,2))

    for i in matrix:
        data = i 
        interval = np.array(st.t.interval(alpha=0.95, df=(matrix.shape[1])-1, loc=np.mean(data), scale=st.sem(data)))
        interval = interval.reshape(1,2)
        cis = np.vstack((cis, interval))

    return cis

In [ ]:
""""This formula is based on page 114 and 115 of 'Queueing Theory' of Ivo Adan and Jacques Resing (2002). 
It is a recursion formula utilizing the relation between the delay probability (W) and the blocking probability (B). 
The recursion is computed, wherafter the delay probability is calculated. 
Little Law's is then used to calculate the mean waiting time.
"""
def theoretical_mmn(rho, mu, n):
    def W(n, rho):
        def B(n, rho):
            B = 1
            for i in range(1, n+1):
                B = rho * B / (i + rho * B)
            return B
        B2 = B(n-1, n* rho)
        return rho * B2 / (1 - rho + rho * B2)
    w = W(n, rho)
    return w / (n * mu) * ( 1 / (1 - rho))

In [ ]:

class DES(object):
    """
    Class named after the Discrete Event Simulation. The program is implemented using SimPy's tutorial for creating events
    Args:
        Simpy.environment
        Rate of arrival; lambda
        Rate of service; mu
        Number of servers; n
        Number of customers; m 
        
    Yields:
        Waiting time for each customer before getting in service
             
    """
    def __init__(self, env, arrival_rate, service_rate, servers, n_samples):
        self.env = env
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.servers = servers
        self.n_samples = n_samples
        self.force = env.event()
        self.action = env.process(self.run())
        self.waiting_times = []
        
    def run(self):
        while True:
            if len(self.waiting_times) > self.n_samples:
                self.force.succeed()
                self.force = self.env.event()
            #customer arrives with exponential inter arrival times (Markovian)
            arrival_time = random.expovariate(self.arrival_rate)
            yield self.env.timeout(arrival_time)
            #service time also has a Markovian distribution, calls service function
            service_time = random.expovariate(self.service_rate)
            self.env.process(self.service(service_time))
            
    # Calculates time that a customer has waited        
    def service(self, service_time):
        before = env.now
        with self.servers.request() as req:
            yield req
            self.waiting_times.append(env.now - before)
            yield self.env.timeout(service_time)

In [ ]:
#-------------------------------------%%%QUESTION 1%%%%------------------------------------------------------------------
"""
To answer question 1, we will make a table with varying rho ranges [0.2, 0.4, 0.6, 0.8, 0.95] for n = 1, 2, 4
"""

In [ ]:
# Lets make a df for the theoretical values

n_samples = 5000
n_servers = np.array([1, 2, 4])
service_rate = 1
rho_range = [0.2, 0.4, 0.6, 0.8, 0.95]
waiting_times_theoretical = np.zeros((len(n_servers), len(rho_range)))

In [ ]:
for i in range(len(n_servers)):
    for j in range(len(rho_range)):
        arrival_rate = rho_range[j] * n_servers[i]
        env = simpy.Environment()
        servers = simpy.Resource(env, capacity = n_servers[i])
        des = DES(env, arrival_rate, (service_rate), servers, n_samples)
        env.run(until = des.force)
        waiting_times_theoretical[i, j] = theoretical_mmn(rho_range[j], service_rate, n_servers[i])


In [ ]:
df = pd.DataFrame(waiting_times_theoretical, columns=[rho_range])
df.to_csv('theoretical.csv', index=False)

In [ ]:
"""
Make a plot which visualizes the difference in using 1, 2 or 4 servers. 
Theoretical values will be included as well to show that the simulation converges 
to the theoretical value when run multiple times.
"""

In [ ]:
runs = 30
n_samples = 200000
n_servers = np.array([1, 2, 4])
rho_range = [0.6, 0.7, 0.8, 0.9, 0.95]
service_rate = 1
waiting_times_mmn = np.zeros((len(n_servers), len(rho_range), runs, n_samples))
waiting_times_theoretical = np.zeros((len(n_servers), len(rho_range)))

In [ ]:
count = 0
for i in range(len(n_servers)):
    for j in range(len(rho_range)):
        for k in range(runs):
            arrival_rate = rho_range[j] * n_servers[i]
            env = simpy.Environment()
            servers = simpy.Resource(env, capacity = n_servers[i])
            des = DES(env, arrival_rate, (service_rate), servers, n_samples)
            env.run(until = des.force)
            waiting_times_mmn[i, j, k, :] = des.waiting_times[:n_samples]
            waiting_times_theoretical[i, j] = theoretical_mmn(rho_range[j], service_rate, n_servers[i])
    count += 1
    print((count/(len(n_servers) * 100), "% done!"))

In [ ]:
mean_waiting_times_runs = np.zeros((len(n_servers), len(rho_range), runs))

for i in range(len(n_servers)):
    for j in range(len(rho_range)):
        for k in range(runs):
            mean_waiting_times_runs[i, j, k] = np.mean(waiting_times_mmn[i, j, k, :])

In [ ]:
mean_waiting_times = np.zeros((len(n_servers), len(rho_range)))

for i in range(len(n_servers)):
    for j in range(len(rho_range)):
        mean_waiting_times[i, j] = np.mean(mean_waiting_times_runs[i, j])

In [ ]:
np.savetxt("MMn_0.6-0.95_runs.csv", mean_waiting_times_runs.reshape(3, 150), delimiter=",")
np.savetxt("MMn_mean_0.6-0.95.csv", mean_waiting_times, delimiter=",")

In [ ]:
mean_waiting_times = genfromtxt('MMn_mean_0.6-0.95.csv', delimiter=',')
mean_waiting_times_runs = genfromtxt('MMn_mean_0.6-0.95_runs.csv', delimiter = ',')

In [ ]:
mean_waiting_times_runs = mean_waiting_times_runs.reshape(3, 5, 30)

In [ ]:
"""

To get a summary of the MMn statistics we will calculate the confidence interval, standard deviation and mean

"""

In [ ]:
#Confidence Interval
cis_n1 = calculate_confidence_interval((mean_waiting_times_runs[0]))
cis_n2 = calculate_confidence_interval((mean_waiting_times_runs[1]))
cis_n3 = calculate_confidence_interval((mean_waiting_times_runs[2]))

cis_n1 = cis_n1[1:]
cis_n2 = cis_n2[1:]
cis_n3 = cis_n3[1:]

In [ ]:
#Standard Deviation
std_mmn_2 = np.zeros((len(n_servers), len(rho_range)))

for i in range(len(n_servers)):
    for j in range(len(rho_range)):
        std_mmn_2[i, j] = np.std(mean_waiting_times_runs[i][j]) 

In [ ]:
for i in range(len(n_servers)):
    if i == 0:
        plt.scatter(rho_range, mean_waiting_times[i], marker = "^", color = 'blue', label = 'Simulation with n = %s' % n_servers[i])
        plt.fill_between(rho_range, mean_waiting_times[i] + 0.2, mean_waiting_times[i] - 0.2,
                         alpha = 0.3, label = "0.05 width tolerance")
    if i == 1:
        plt.scatter(rho_range, mean_waiting_times[i], marker = "o", color ='green', label = 'Simulation with n = %s' % n_servers[i])
        plt.fill_between(rho_range, mean_waiting_times[i] + 0.2, mean_waiting_times[i] - 0.2,
                         alpha = 0.3, label = "0.05 width tolerance")
    if i == 2:
        plt.scatter(rho_range, mean_waiting_times[i], marker = "8", color = 'red', label = 'Simulation with n = %s' % n_servers[i])
        plt.fill_between(rho_range, mean_waiting_times[i] + 0.2, mean_waiting_times[i] - 0.2,
                         alpha = 0.3, label = "0.05 width tolerance")
        
for i in range(len(n_servers)):
    plt.plot(rho_range, waiting_times_theoretical[i], label = 'Theoretical with n = %s' % n_servers[i])

plt.xlabel("Utilization rate " r'$\rho$', fontsize = 15)
plt.ylabel("E(W)", fontsize = 15)
plt.title("Waiting times for M/M/n Queuing simulation", fontsize  = 15)
plt.tick_params(axis='both', which='major', labelsize=13)
plt.legend(fontsize=12)
plt.savefig("MMn_EW.png")

In [ ]:
"""
Lets investigate the influence of rho while increasing the sample size 
on the mean waiting time and on the standard deviation
"""

In [ ]:
runs = 30
#This is our sample range
n_samples = [100, 500, 1000, 5000, 10000, 20000, 50000, 75000, 100000]
n_servers = np.array([1, 2, 4])
rho_range = [0.6, 0.7, 0.8, 0.9, 0.95]
service_rate = 1
arrival_rate = 0
waiting_times_mmn_2 = []
#waiting_times_theoretical = np.zeros((len(rho_range), num_runs, len(n_servers)))

In [ ]:
for x in range(len(n_samples)):
    count = 0
    print("now starting with sample {}".format(n_samples[x]))
    waiting_times_mmn_2_temp = np.zeros((len(n_servers), len(rho_range), runs, n_samples[x]))
    for i in range(len(n_servers)):
        for j in range(len(rho_range)):
            for k in range(runs):
                arrival_rate = n_servers[i] * rho_range[j] 
                env = simpy.Environment()
                servers = simpy.Resource(env, capacity = n_servers[i])
                des = DES(env, arrival_rate, (service_rate), servers, n_samples[x])
                env.run(until = des.force)
                waiting_times_mmn_2_temp[i, j, k, :] = des.waiting_times[:n_samples[x]]
                #waiting_times_theoretical[i, j, k] = theoretical_mmn(rho_range[j], service_rate, n_servers[i])
        count += 1
        print(count/len(n_servers) * 100, "% done!")
                
    waiting_times_mmn_2.append(waiting_times_mmn_2_temp)

In [ ]:
#Calculating the mean for each of the number of samples in the 30 runs

mean_waiting_times_mmn_2 = np.zeros((len(n_samples),len(n_servers), len(rho_range), runs))

for x in range(len(n_samples)):
    for i in range(len(n_servers)):
        for j in range(len(rho_range)):
            for k in range(runs):
                mean_waiting_times_mmn_2[x, i, j, k] = np.mean(waiting_times_mmn_2[x][i][j][k]) 
        

In [ ]:
#Calculates the mean of the 30 runs to get 1 mean value per number of servers with the corresponding rho value

mean_waiting_times_mmn_2_1 = np.zeros((len(n_samples),len(n_servers), len(rho_range)))

for x in range(len(n_samples)):
    for i in range(len(n_servers)):
        for j in range(len(rho_range)):
            mean_waiting_times_mmn_2_1[x, i, j] = np.mean(mean_waiting_times_mmn_2[x,i,j])

In [ ]:
#For convience we split the data for each server number

mean_n1 = mean_waiting_times_mmn_2_1[:, 0, :]
mean_n2 = mean_waiting_times_mmn_2_1[:, 1, :]
mean_n4 = mean_waiting_times_mmn_2_1[:, 2, :]

In [ ]:
#Plots the number of customers vs the mean waiting time for varying rho values [0.6, 0.7, 0.8, 0.9, 0.95]

fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig.set_size_inches(8, 10, forward=True)
ax1.plot(n_samples, mean_n1, label = rho_range)
ax1.set_title("n = 1", fontsize  = 15)
ax1.set_ylabel("E(W)", fontsize  = 15)
ax1.set_tick_params(axis='both', which='major', labelsize=13)
ax1.legend()
ax2.plot(n_samples, mean_n2, label = rho_range)
ax2.set_title("n = 2", fontsize  = 15)
ax2.set_ylabel("E(W)", fontsize  = 15)
ax2.set_tick_params(axis='both', which='major', labelsize=13)
ax2.legend()
ax3.plot(n_samples, mean_n4, label = rho_range)
ax3.set_title("n = 4", fontsize  = 15)
ax3.set_xlabel("Number of customers", fontsize = 15)
ax3.set_ylabel("E(W)", fontsize  = 15)
ax3.set_tick_params(axis='both', which='major', labelsize=13)
ax3.legend()
fig.title("Waiting times for M/M/n Queuing simulation", fontsize  = 15)
fig.savefig('MMN_mean.png')


In [ ]:
#Calculates the standard deviation

std_mmn_2 = np.zeros((len(n_samples),len(n_servers), len(rho_range)))

for x in range(len(n_samples)):
    for i in range(len(n_servers)):
        for j in range(len(rho_range)):
            std_mmn_2[x, i, j] = np.std(mean_waiting_times_mmn_2[x][i][j]) 

In [ ]:
std_n1 = std_mmn_2[:, 0, :]
std_n2 = std_mmn_2[:, 1, :]
std_n4 = std_mmn_2[:, 2, :]

In [ ]:
#Let's plot the STD vs Number of customers

fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig.set_size_inches(8, 10, forward=True)
ax1.plot(n_samples, std_n1, label = rho_range)
ax1.set_title("n = 1", fontsize  = 15)
ax1.set_ylabel("Std", fontsize  = 15)
ax1.set_tick_params(axis='both', which='major', labelsize=13)
ax1.legend()
ax2.plot(n_samples, std_n2, label = rho_range)
ax2.set_title("n = 2", fontsize  = 15)
ax2.set_ylabel("Std", fontsize  = 15)
ax2.set_tick_params(axis='both', which='major', labelsize=13)
ax2.legend()
ax3.plot(n_samples, std_n4, label = rho_range)
ax3.set_title("n = 4", fontsize  = 15)
ax3.set_xlabel("Number of customers", fontsize  = 15)
ax3.set_ylabel("Std", fontsize  = 15)
ax3.set_tick_params(axis='both', which='major', labelsize=13)
ax3.legend()
fig.savefig('MMN_std.png')

In [ ]:
# -------------------------- %%% Code obtained from question3 and question 4 file%%% ------------------------------
# This will produce a system compatible for comparing to the other systems like M/D/n, M/H/n, and M/M/n with SJF
n_samples = 200000
n_servers = np.array([1,2,4])
steps = 10
arrival_rate = n_servers
p_min = 0.5
p_max = 0.95
p_range = np.linspace(p_min, p_max, steps)
service_rate = (1 / p_range)
runs = 25

waiting_times_mmn3 = np.zeros((3, steps, n_samples))
waiting_times_mmn3_stacked = np.zeros((1, runs))
count = 0
for i in range(len(n_servers)):
    for j in tqdm(range(steps), desc=f'calculate waiting times for n_server {n_servers[i]}'):
        waiting_times_mmn3_stacked_temp = np.zeros((1, n_samples))
        for k in range(runs):
            env = simpy.Environment()
            servers1 = simpy.PriorityResource(env, capacity=n_servers[i])
            waiting_times = []
            setup1 = DES(env, arrival_rate[i], service_rate[j], servers1, n_samples)
            env.run(until=setup1.force)            
            waiting_times_mmn3_stacked_temp = np.vstack((waiting_times_mmn3_stacked_temp, setup1.waiting_times[:n_samples]))
        appending = np.mean(waiting_times_mmn3_stacked_temp[1:], axis = 1)
        apend = appending.reshape(1, appending.shape[0])
        waiting_times_mmn3_stacked = np.vstack((waiting_times_mmn3_stacked,apend))
        
waiting_times_mmn3_stacked = waiting_times_mmn3_stacked[1:]
np.savetxt("MMN_0.5_0.95.csv", waiting_times_mmn3_stacked, delimiter=",")
#%%

In [ ]:
waiting_times_MMn_stacked = genfromtxt('MMN_0.5_0.95.csv', delimiter=',')
relavant_std_MM = np.std(waiting_times_MMn_stacked, axis = 1)

In [ ]:
#Plots the standard deviation vs rho range for n=1, n=2, and n=4
n_servers = np.array([1, 2, 4])
for i in n_servers:
    if i == 1:
        plt.plot(p_range, relavant_std_MM[0:i*10], marker = "^",
                 label = 'standard deviation for {} server(s)'.format(i), linewidth = 2, markersize = 8)
    elif i == 2:
        plt.plot(p_range, relavant_std_MM[((i)*5):((i+2)*5)], marker = "o", linestyle = "--",
                 label = 'standard deviation for {} server(s)'.format(i), linewidth = 2, markersize = 8)
    elif i == 4:
        plt.plot(p_range, relavant_std_MM[((i)*5):((i+2)*5)], marker = "8", linestyle  = "-.",
                 label = 'standard deviation for {} server(s)'.format(i), linewidth = 2, markersize = 8)
plt.legend(fontsize = 13)
plt.xlabel("Utilization rate " r'$\rho$', fontsize = 15)
plt.ylabel(r'$S[\bar{W}]$', fontsize = 15)
plt.title("Standard deviation for MM/1 - MM/n Queuing simulation", fontsize  = 15)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.savefig("MMN_std_vs_rho")